In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

TOP_POSTER_LIMIT = 10
DOMAIN_LIMIT = 25
REPLIES_THRESHOLD = 150

df = pd.read_csv("data/latest-articles.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"], unit='s')
df["title"] = df["title"].astype("category")
df["posted_by"] = df["posted_by"].astype("category")
df["source_url"] = df["source_url"].astype("category")

In [2]:
# A thread is considered sourced if it has a URL link.
sourced = df[df.source_url != ""]

# This is not entirely accurate since some vanity posts have links.
vanity = df[df.source_url == ""]

In [3]:
def extract_domain(url) -> str:
    return (
        url.
            replace("www.","").
            replace("http://", "").
            replace("https://", "").
            split("/")[0]
    )


sourced.insert(len(sourced.columns)-1, "domain", (
    sourced.
        source_url.
        apply(extract_domain).
        astype("category")
))

<Axes: xlabel='count', ylabel='domain'>

In [4]:
# Draws a chart that shows where discussion for a particular thread originated from.
data = sourced[["source_url", "domain"]].drop_duplicates()
top_sourced_domains = data.domain.value_counts().iloc[:DOMAIN_LIMIT].index
sns.countplot(
    y="domain",
    data=data,
    order=top_sourced_domains,
)

<Axes: xlabel='count', ylabel='posted_by'>

In [5]:
# Draws a chart that shows who posted a topic to be discussed that has a URL source.
data = sourced[["source_url", "posted_by"]].drop_duplicates()
top_sourced_posters = data.posted_by.value_counts().iloc[:TOP_POSTER_LIMIT].index
sns.countplot(
    y="posted_by",
    data=data,
    order=top_sourced_posters,
)

In [6]:
# Comment this if you don't want to see everything.
pd.set_option("display.max_rows", None, "display.max_columns", None)

# Draws a table that shows which freepers posted topics from a sourced link that originated
# from a particular domain.
who_is_posting_from_what = (
    sourced[["posted_by", "source_url", "domain"]].
    query("posted_by in @top_sourced_posters and domain in @top_sourced_domains").
    drop_duplicates().
    groupby(["domain", "posted_by"])
)
who_is_posting_from_what[["domain"]].describe()

/tmp/ipykernel_1283687/2178239382.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groupby(["domain", "posted_by"])


domain         \
                                            count unique   
domain               posted_by                             
americanthinker.com  MtnClimber                66      1   
                     SeekAndFind               31      1   
                     george76                   2      1   
apnews.com           SeekAndFind                1      1   
                     E. Pluribus Unum           7      1   
                     RandFan                    3      1   
                     Red Badger                 1      1   
                     marcusmaximus              3      1   
babylonbee.com       george76                   2      1   
                     E. Pluribus Unum           3      1   
                     marcusmaximus              1      1   
                     Morgana                    3      1   
bbc.com              MtnClimber                 1      1   
                     Red Badger                 4      1   
                     marcusmaximus              2      1   
breitbart.com        MtnClimber                 1      1   
                     george76                   2      1   
                     E. Pluribus Unum           2      1   
                     Red Badger                 6      1   
                     marcusmaximus              1      1   
                     Morgana                   28      1   
                     ChicagoConservative27    260      1   
dailymail.co.uk      SeekAndFind                2      1   
                     george76                   3      1   
                     E. Pluribus Unum           1      1   
                     RandFan                    6      1   
                     Red Badger                 9      1   
                     marcusmaximus              9      1   
                     Morgana                  122      1   
                     Libloather                 4      1   
foxnews.com          george76                  11      1   
                     E. Pluribus Unum           2      1   
                     RandFan                    1      1   
                     Red Badger                 4      1   
                     marcusmaximus              2      1   
                     Morgana                    1      1   
                     ChicagoConservative27      2      1   
                     Libloather                25      1   
justthenews.com      E. Pluribus Unum          14      1   
                     Red Badger                 4      1   
msn.com              SeekAndFind                8      1   
                     E. Pluribus Unum           2      1   
                     RandFan                    7      1   
                     Red Badger                 1      1   
                     marcusmaximus              3      1   
                     ChicagoConservative27      2      1   
newsbusters.org      Morgana                    1      1   
newsweek.com         MtnClimber                 1      1   
                     george76                   1      1   
                     E. Pluribus Unum           3      1   
                     Red Badger                 3      1   
                     marcusmaximus              6      1   
                     ChicagoConservative27      1      1   
                     Libloather                 3      1   
nypost.com           MtnClimber                 3      1   
                     SeekAndFind                1      1   
                     george76                  17      1   
                     E. Pluribus Unum           3      1   
                     RandFan                    1      1   
                     Red Badger                 2      1   
                     marcusmaximus             19      1   
                     Morgana                    2      1   
                     ChicagoConservative27     43      1   
                     Libloather               118      1   
nytimes.com          E.

In [7]:
# Display which posts got the most replies before
# they aged out from the front page.

(sourced[['timestamp', 'title', 'replies']].
     loc[sourced.replies > REPLIES_THRESHOLD].
     groupby('title').
     max().
     sort_values('replies', ascending=False).
     dropna())

/tmp/ipykernel_1283687/1921096508.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groupby('title').


,timestamp,replies
title,,
Small aircraft crashes into Potomac River near Reagan National Airport,2025-01-30 11:55:48,628.0
Steve Bannon: “I will have Elon Musk run out of here by Inauguration Day”,2025-01-13 09:38:57,283.0
President Trump speaks after plane crash,2025-01-30 18:06:42,216.0
One of 3 soldiers aboard Black Hawk chopper involved in midair collision identified as former White House aide (prior unidentified pilot),2025-02-02 00:25:37,211.0
Female Pilot Rebecca Lobach’s Entire Social Media Scrubbed before Army Released Her Name – So What Are They Hiding?,2025-02-02 17:04:22,204.0
Supreme Court leans in favor of state-enforced age limits on porn websites,2025-01-16 10:52:22,201.0
"New Fire in LA, Hollywood Hills!",2025-01-09 05:48:08,199.0
Female Pilot in Crash Trump Blamed on DEI Was Top 20% Army Cadet,2025-02-02 22:12:55,188.0
Hegseth confirmed 51-50.,2025-01-25 03:54:12,176.0
